In [1]:
from Util import get_openai_api_key
OPEN_API_KEY=get_openai_api_key()

In [2]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [7]:
import pandas as pd
data_item= pd.read_csv("email_messages.csv")
new_data= data_item.to_string(index=False)

In [8]:
from autogen import AssistantAgent

email_detector_agent=AssistantAgent(
    name="email_detector_agent",
    system_message=f""" 
    You are an intelligent data processing assistant. You are given a dataset containing email messages with columns: Email_ID, Sender_Email, Subject, Content, and Label. Your task is to analyze each email and detect whether it is a "Spam" or "Verified" message based on suspicious keywords or patterns like fake prizes, urgent actions, or offers.

    Your primary goal is to filter and return only the rows where the email is classified as "Verified" in the Label column. Ignore or discard any row marked as "Spam".
    
    Output only the filtered dataset with the following columns: Email_ID, Sender_Email, Subject, Content, Label.

    Hers is the dataset:

    {new_data}

    """,
    human_input_mode="NEVER",
    llm_config=llm_config

)

In [9]:
csv_writer_agent = AssistantAgent(
    name="csv_writer_agent",
    system_message="""
You are a CSV writing assistant.

You will receive a filtered dataset (as text table or CSV string).
Task:
- Convert it into pandas DataFrame.
- Generate Python code to save it into 'verified_emails.csv'.
Output only Python code inside a code block.
""",
    llm_config=llm_config,
)

In [11]:
from autogen import ConversableAgent
user_agent = ConversableAgent(
    name="user_agent",
    human_input_mode="NEVER"
)

In [12]:
from autogen import GroupChat,GroupChatManager

group_chat= GroupChat(
    agents=[
        email_detector_agent,
        csv_writer_agent,
        user_agent
    ],
    messages=[],
    max_round=5
)
manager= GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

In [14]:
print("📬 Starting Agentic Workflow with GroupChat...\n")

# Kick off the conversation from user_agent
results=user_agent.initiate_chat(
    manager,
    message=f"""Here is the dataset of email messages:\n\n{new_data}\n\nPlease filter only the verified emails and convert the result to a CSV file."""
)

📬 Starting Agentic Workflow with GroupChat...

user_agent (to chat_manager):

Here is the dataset of email messages:

 Email_ID             Sender_Email                 Subject                                 Content    Label
        1      lottery@fakewin.com        Congratulations!     You've won $10,000. Click to claim!     Spam
        2       support@amazon.com            Your receipt   Your Amazon order #12345 has shipped. Verified
        3     freegift@scammer.net  Claim your free iPhone                 Only 1 left! Enter now!     Spam
        4     updates@linkedin.com  New connection request   You have a new connection from Sarah. Verified
        5     offers@cheapmeds.com         Buy meds online              Get Viagra at 90% off now!     Spam
        6      no-reply@github.com          Security alert A login was detected from a new device. Verified
        7 winbig@casino-online.biz            Win big now!     Deposit now and get 100 free spins!     Spam
        8  notific

In [15]:
data = {
    'Email_ID': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    'Sender_Email': ['support@amazon.com', 'updates@linkedin.com', 'no-reply@github.com', 'notifications@slack.com', 'newsletter@medium.com', 'billing@netflix.com', 'info@zoom.us', 'support@spotify.com', 'info@coursera.org', 'alerts@icloud.com'],
    'Subject': ['Your receipt', 'New connection request', 'Security alert', 'New message alert', 'Top stories today', 'Your payment is due', 'Meeting reminder', 'Password reset', 'Course completion', 'New sign-in alert'],
    'Content': ['Your Amazon order #12345 has shipped.', 'You have a new connection from Sarah.', 'A login was detected from a new device.', 'You received a new message in #general.', 'Here are your daily Medium highlights.', 'Your next billing cycle starts soon.', 'Your meeting starts in 15 minutes.', 'Reset your password using this link.', 'You’ve completed your course on Python.', 'Your Apple ID was used to sign in.'],
    'Label': ['Verified'] * 10
}

df = pd.DataFrame(data)

In [16]:
df.to_csv('verified_emails.csv', index=False)